In [5]:
import numpy as np
import pandas as pd
import math

In [6]:
#The original paper square rooted the denominator
#I assume this is for normalized z scores, for the time being it is left as is

def Tsimilarity(x,y):
    return np.dot(x,y)/(pow(np.linalg.norm(x), 2) + pow(np.linalg.norm(y), 2) - abs(np.dot(x,y)))

In [7]:
def zScoreNorm(X):
    mean = X.stack().mean()
    std_dev = X.stack().std()
    return (X - mean)/std_dev

In [8]:
# X and Y are two networks
# the networkGen function iterates over the rows and columns, producing the T similarity
# of each node, to the other, populating the matrix
def networkGen(X, Y):

    df = pd.DataFrame(index=X.columns, columns = Y.index)
    
    for x in range(len(X.columns)):
        for y in range(len(Y)): 
            df.iat[x,y] = Tsimilarity(X.iloc[:,x], Y.iloc[y])

    return df

def regNetworkGen(X, Y):
    df = pd.DataFrame(index=X.index, columns = Y.index)
    
    for x in range(len(X)):
        for y in range(len(Y)):
            df.iat[x,y] = Tsimilarity(X.iloc[x], Y.iloc[y])

    return df

def coregNetworkGen(X, Y):
    df = pd.DataFrame(index=X.columns, columns = Y.columns)
    
    for x in range(len(X.columns)):
        for y in range(len(Y.columns)):
            df.iat[x,y] = Tsimilarity(X.iloc[:,x], Y.iloc[:,y])

    return df


In [9]:
# Network Averaging takes the responsability and availability
# and average the values to create W-bar
def networkAverage(X, Y):

    df_avg = (X + Y)/2

    return df_avg
    
# Finally network updating is the creates a new epoch, offset by 
# some aribtrary alpha
def networkUpdate(W, W_tilde, alpha):
    return W*(1 - alpha) + W_tilde*(alpha)
    
# In the hyper loop of the program, we find the mean of W - W-bar network 
# if it is > 0.000001 or < 2 we terminate the program

In [14]:
#effector-affected frame
Initial = {
    "JPM": [0.551967, 0.420968, 0.157405, 0.230747, 0.390506],
    "BAC": [0.407512, 0.149711, 0.092709, 0.226999, 0.373366],
    "WFC": [0.523636, 0.184994, 0.049343, 0.273286, 0.310993],
    "GS": [0.726089, 0.488475, 0.263886, 0.330813, 0.20736],
    "C": [0.841646, 0.812474, 0.641967, 0.268008, 0.5194]
}

# Define the index
index = ["AAPL", "MSFT", "GOOGL", "AMZN", "META"]

W = pd.DataFrame(Initial, index=index)
W = zScoreNorm(W)

Tech = {
    "AAPL": [None, 0.684288, 0.726008, 0.944709, 0.889102],
    "MSFT": [0.684288, None, 0.446464, 0.985835, 0.775893],
    "GOOGL": [0.726008, 0.446464, None, 0.966784, 0.775873],
    "AMZN": [0.944709, 0.985835, 0.966784, None, 0.22929],
    "META": [0.889102, 0.775893, 0.775873, 0.22929, None]
}

# T(ech) is the equivelent to P in the original paper
T = pd.DataFrame(Tech, index=Tech.keys()).fillna(0)
T = zScoreNorm(T)


Finance = {
    "JPM": [None, 0.725199, 0.207714, 0.182977, 0.752395],
    "BAC": [0.725199, None, 0.125959, 0.066353, 0.967194],
    "WFC": [0.207714, 0.125959, None, 0.008894, 0.929191],
    "GS": [0.182977, 0.066353, 0.008894, None, 0.452338],
    "C": [0.752395, 0.967194, 0.929191, 0.452338, None]
}

#F(inance) is the analog for the Co-regulatory network in the original paper
F = pd.DataFrame(Finance, index=Finance.keys()).fillna(0)
F = zScoreNorm(F)

#Fix variance in reg and coreg: only consider upper triangular of T and F
#Supplemental methods to calculate diagonal of reg and coreg
#Force symmetry 

In [15]:
H = 0.5
alpha = 0.01
iteration = 0

while(H > 0.00001 and H < 2):
    R = networkGen(W, T)

    A = networkGen(F, W)
    
    W_tilde = networkAverage(R, A)
    W_tilde = W_tilde.transpose()
    
    W = networkUpdate(W, W_tilde, alpha)
    
    T_tilde = regNetworkGen(W, W)
    
    F_tilde = coregNetworkGen(W,W)

    T = networkUpdate(T, T_tilde, alpha)

    F = networkUpdate(F, F_tilde, alpha)

    H = np.mean(abs(W - W_tilde).values)

    iteration = iteration + 1
    print(H)

print(iteration)
print(W)
print(H)

1.023158281603143
1.0113020560144532
0.9995341846418536
0.9878531833390017
0.9762575753923748
0.964745891917499
0.9533166722743206
0.9419684645395431
0.9306998260396255
0.9195093239482576
0.9083955359522373
0.897357050989759
0.8863924700651884
0.8755004071444524
0.8646794901351733
0.8539283619556728
0.8432456816969038
0.8326301258812694
0.8220803898221355
0.8115951890876407
0.8011732610721269
0.7908133666781815
0.7805142921118651
0.7702748507931956
0.7600938853833873
0.7499702699296461
0.7399029121275607
0.729890755700241
0.7199327828923721
0.7100280170762562
0.7001747950535524
0.6903712191968795
0.680616334849755
0.6709092382209408
0.6612490759452716
0.6516350482847681
0.6420664124259113
0.6325424858631866
0.623062649857455
0.6136263529561227
0.604233103570948
0.5948824590490587
0.5855740940620326
0.5763077688710433
0.5670833292608158
0.5579007100366181
0.5487599388568426
0.5396599074089412
0.530599419371556
0.5215787076813737
0.5125981134592233
0.503657252806853
0.49475522825697477
0